In [3]:
import sys
sys.path.insert(0, "../src")

In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [5]:
import gc
from pathlib import Path
from tqdm.notebook import tqdm

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import RandomSampler, SequentialSampler
import pytorch_lightning as pl

import transformers

import optim
import loss
from utils import visualize, radar2precipitation, seed_everything

<IPython.core.display.Javascript object>

# U-Net

## Config

In [24]:
args = dict(
    seed=42,
    dams=(6071, 6304, 7026, 7629, 7767, 8944, 11107),
    train_folds_csv=Path("../input/train_folds.csv"),
    train_data_path=Path("../input/train-128"),
    test_data_path=Path("../input/test-128"),
    model_dir=Path("../models"),
    rng=255.0,
    num_workers=4,
    gpus=1,
    lr=1e-4,
    max_epochs=50,
    batch_size=256,
    precision=16,
    optimizer="adamw",
    scheduler="cosine",
    accumulate_grad_batches=1,
    gradient_clip_val=5.0,
)

<IPython.core.display.Javascript object>

## Model

### Layers

#### Basic

In [7]:
class BasicBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        assert in_ch == out_ch
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
        )

    def forward(self, x):
        return x + self.net(x)

<IPython.core.display.Javascript object>

#### Encoder

In [8]:
class DownBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.id_conv = nn.Conv2d(in_ch, out_ch, kernel_size=1, stride=2)
        self.net = nn.Sequential(
            nn.BatchNorm2d(in_ch),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(in_ch),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
        )

    def forward(self, x):
        residual = x
        residual = self.id_conv(residual)
        x = self.net(x)
        return residual + x, x

<IPython.core.display.Javascript object>

In [9]:
class Encoder(nn.Module):
    def __init__(self, chs=[4, 64, 128, 256, 512, 1024]):
        super().__init__()
        self.blocks = nn.ModuleList(
            [DownBlock(chs[i], chs[i + 1]) for i in range(len(chs) - 1)]
        )
        self.basic = BasicBlock(chs[-1], chs[-1])

    def forward(self, x):
        feats = []
        for block in self.blocks:
            x, feat = block(x)
            feats.append(feat)
        x = self.basic(x)
        feats.append(x)
        return feats

<IPython.core.display.Javascript object>

#### Decoder

In [10]:
class UpBlock(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=False):
        super().__init__()
        self.id_conv = nn.ConvTranspose2d(
            in_ch + in_ch, out_ch, kernel_size=2, stride=2
        )
        layers = []
        if bilinear:
            layers.append(nn.Upsample(scale_factor=2, mode="nearest"))
        else:
            layers.append(
                nn.ConvTranspose2d(in_ch + in_ch, out_ch, kernel_size=2, stride=2)
            )
        layers.extend(
            [
                nn.BatchNorm2d(out_ch),
                nn.LeakyReLU(inplace=True),
                nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
                nn.BatchNorm2d(out_ch),
                nn.LeakyReLU(inplace=True),
                nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
            ]
        )
        self.block = nn.Sequential(*layers)

    def forward(self, x, feat):
        x = torch.cat([x, feat], dim=1)
        residual = x
        residual = self.id_conv(residual)
        x = self.block(x)
        return x + residual

<IPython.core.display.Javascript object>

In [11]:
class Decoder(nn.Module):
    def __init__(self, chs=[1024, 512, 256, 128, 64]):
        super().__init__()
        self.blocks = nn.ModuleList(
            [UpBlock(chs[i], chs[i + 1]) for i in range(len(chs) - 1)]
        )

    def forward(self, x, feats):
        for block, feat in zip(self.blocks, feats):
            x = block(x, feat)
        return x

<IPython.core.display.Javascript object>

In [12]:
# x = torch.randn(3, 4, 128, 128)
# encoder = Encoder()
# feats = encoder(x)
# for feat in feats:
#     print(feat.shape)

<IPython.core.display.Javascript object>

In [13]:
# decoder = Decoder()
# x = torch.randn(3, 1024, 4, 4)
# feats = list(reversed(feats))[1:]
# decoder(x, feats).shape

<IPython.core.display.Javascript object>

In [14]:
class UNet(pl.LightningModule):
    def __init__(
        self,
        lr=args["lr"],
        enc_chs=[4, 64, 128, 256, 512, 1024],
        dec_chs=[1024, 512, 256, 128, 64],
        num_train_steps=None,
    ):
        super().__init__()
        self.lr = lr
        self.num_train_steps = num_train_steps
        #         self.criterion = nn.SmoothL1Loss()
        self.criterion = nn.L1Loss()

        self.tail = BasicBlock(4, enc_chs[0])
        self.encoder = Encoder(enc_chs)
        self.decoder = Decoder(dec_chs)
        self.head = nn.Sequential(
            nn.ConvTranspose2d(dec_chs[-1], 32, kernel_size=2, stride=2, bias=False),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(32, 1, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.tail(x)
        feats = self.encoder(x)
        feats = feats[::-1]
        x = self.decoder(feats[0], feats[1:])
        x = self.head(x)

        return x

    def shared_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)

        return loss, y, y_hat

    def training_step(self, batch, batch_idx):
        loss, y, y_hat = self.shared_step(batch, batch_idx)
        self.log("train_loss", loss)
        for i, param_group in enumerate(self.optimizer.param_groups):
            self.log(f"lr/lr{i}", param_group["lr"])

        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        loss, y, y_hat = self.shared_step(batch, batch_idx)

        return {"loss": loss, "y": y.detach(), "y_hat": y_hat.detach()}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
        self.log("val_loss", avg_loss)

        crop = T.CenterCrop(120)

        y = torch.cat([x["y"] for x in outputs])
        y = crop(y)
        y = y.detach().cpu().numpy()
        y = y.reshape(-1, 120 * 120)

        y_hat = torch.cat([x["y_hat"] for x in outputs])
        y_hat = crop(y_hat)
        y_hat = y_hat.detach().cpu().numpy()
        y_hat = y_hat.reshape(-1, 120 * 120)

        y = args["rng"] * y[:, args["dams"]]
        y = y.clip(0, 255)
        y_hat = args["rng"] * y_hat[:, args["dams"]]
        y_hat = y_hat.clip(0, 255)

        y_true = radar2precipitation(y)
        y_true = np.where(y_true >= 0.1, 1, 0)
        y_pred = radar2precipitation(y_hat)
        y_pred = np.where(y_pred >= 0.1, 1, 0)

        y *= y_true
        y_hat *= y_true
        mae = metrics.mean_absolute_error(y, y_hat)
        self.log("mae", mae)

        tn, fp, fn, tp = metrics.confusion_matrix(
            y_true.ravel(), y_pred.ravel()
        ).ravel()
        csi = tp / (tp + fn + fp)
        self.log("csi", csi)

        comp_metric = mae / (csi + 1e-12)
        self.log("comp_metric", comp_metric)

        print(
            f"Epoch {self.current_epoch} | MAE/CSI: {comp_metric} | MAE: {mae} | CSI: {csi} | Loss: {avg_loss}"
        )

    def configure_optimizers(self):

        # Optimizer
        if args["optimizer"] == "adam":
            self.optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        elif args["optimizer"] == "adamw":
            self.optimizer = transformers.AdamW(self.parameters(), lr=self.lr)
        elif args["optimizer"] == "adagrad":
            self.optimizer = torch.optim.Adagrad(self.parameters(), lr=self.lr)
        elif args["optimizer"] == "radam":
            self.optimizer = optim.RAdam(self.parameters(), lr=self.lr)
        elif args["optimizer"] == "ranger":
            optimizer = optim.RAdam(self.parameters(), lr=self.lr)
            self.optimizer = optim.Lookahead(optimizer)

        # Scheduler
        if args["scheduler"] == "cosine":
            self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                self.optimizer, T_max=self.num_train_steps
            )
            return [self.optimizer], [{"scheduler": self.scheduler, "interval": "step"}]
        elif args["scheduler"] == "step":
            self.scheduler = torch.optim.lr_scheduler.StepLR()
        elif args["scheduler"] == "plateau":
            self.scheduler = torch.optim.lr_scheduler.ReduceLR()
        else:
            self.scheduler = None
            return [self.optimizer]

<IPython.core.display.Javascript object>

In [18]:
# m = UNet()
# x = torch.randn(3, 4, 128, 128)
# m(x).shape

<IPython.core.display.Javascript object>

## Dataset

In [19]:
class NowcastingDataset(torch.utils.data.Dataset):
    def __init__(self, paths, test=False):
        self.paths = paths
        self.test = test

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        data = np.load(path)

        x = data[:, :, :4]
        x = x / args["rng"]
        x = x.astype(np.float32)
        x = torch.tensor(x, dtype=torch.float)
        x = x.permute(2, 0, 1)
        if self.test:
            return x
        else:
            y = data[:, :, 4]

            precipitation = radar2precipitation(y)

            label = np.zeros(y.shape)
            label[precipitation >= 0.1] += 1
            label[precipitation >= 1.0] += 1
            label[precipitation >= 2.5] += 1
            label = torch.tensor(label, dtype=torch.long)
            label = label.unsqueeze(0)

            y = y / args["rng"]
            y = y.astype(np.float32)
            y = torch.tensor(y, dtype=torch.float)
            y = y.unsqueeze(-1)
            y = y.permute(2, 0, 1)

            return x, y, label

<IPython.core.display.Javascript object>

In [20]:
fold = 3
df = pd.read_csv(args["train_folds_csv"])
train_df = df[df.fold != fold]
train_paths = [args["train_data_path"] / fn for fn in train_df.filename.values]
dataset = NowcastingDataset(train_paths)
idx = np.random.randint(len(dataset))
x, y = dataset[idx]

ValueError: too many values to unpack (expected 2)

<IPython.core.display.Javascript object>

In [21]:
class NowcastingDataModule(pl.LightningDataModule):
    def __init__(
        self,
        train_df=None,
        val_df=None,
        batch_size=args["batch_size"],
        num_workers=args["num_workers"],
    ):
        super().__init__()
        self.train_df = train_df
        self.val_df = val_df
        self.batch_size = batch_size
        self.num_workers = num_workers

    def setup(self, stage="train"):
        if stage == "train":
            train_paths = [
                args["train_data_path"] / fn for fn in self.train_df.filename.values
            ]
            val_paths = [
                args["train_data_path"] / fn for fn in self.val_df.filename.values
            ]
            self.train_dataset = NowcastingDataset(train_paths)
            self.val_dataset = NowcastingDataset(val_paths)
        else:
            test_paths = list(sorted(args["test_data_path"].glob("*.npy")))
            self.test_dataset = NowcastingDataset(test_paths, test=True)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            sampler=RandomSampler(self.train_dataset),
            pin_memory=True,
            num_workers=self.num_workers,
            drop_last=True,
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_dataset,
            batch_size=2 * self.batch_size,
            sampler=SequentialSampler(self.val_dataset),
            pin_memory=True,
            num_workers=self.num_workers,
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_dataset,
            batch_size=2 * self.batch_size,
            sampler=SequentialSampler(self.test_dataset),
            pin_memory=True,
            num_workers=self.num_workers,
        )

<IPython.core.display.Javascript object>

## Train

In [22]:
seed_everything(args["seed"])
pl.seed_everything(args["seed"])

df = pd.read_csv(args["train_folds_csv"])

for fold in range(3, 5):
    train_df = df[df.fold != fold]
    val_df = df[df.fold == fold]

    datamodule = NowcastingDataModule(
        train_df, val_df, batch_size=args["batch_size"], num_workers=args["num_workers"]
    )
    datamodule.setup()

    num_train_steps = (
        int(
            np.ceil(
                len(train_df) // args["batch_size"] / args["accumulate_grad_batches"]
            )
        )
        * args["max_epochs"]
    )

    model = UNet(num_train_steps=num_train_steps)

    trainer = pl.Trainer(
        gpus=args["gpus"],
        max_epochs=args["max_epochs"],
        precision=args["precision"],
        progress_bar_refresh_rate=50,
        #         accumulate_grad_batches=args["accumulate_grad_batches"],
        #         gradient_clip_val=args["gradient_clip_val"],
        auto_lr_find=True,
    )

    # learning rate finder
    #     lr_finder = trainer.tuner.lr_find(model, datamodule=datamodule)
    #     fig = lr_finder.plot(suggest=True)
    #     fig.show()

    trainer.fit(model, datamodule)
    trainer.save_checkpoint(
        args["model_dir"]
        / f"unet_fold{fold}_bs{args['batch_size']}_epoch{args['max_epochs']}_{args['optimizer']}_{args['scheduler']}.ckpt"
    )

    del datamodule, model, trainer
    gc.collect()
    torch.cuda.empty_cache()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.

  | Name      | Type       | Params
-----------------------------------------
0 | criterion | L1Loss     | 0     
1 | tail      | BasicBlock | 300   
2 | encoder   | Encoder    | 25 M  
3 | decoder   | Decoder    | 17 M  
4 | head      | Sequential | 8 K   


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/isleof/.pyenv/versions/miniconda3-latest/envs/torch2/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-374e52f4655e>", line 41, in <module>
    trainer.fit(model, datamodule)
  File "/home/isleof/.pyenv/versions/miniconda3-latest/envs/torch2/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py", line 440, in fit
    results = self.accelerator_backend.train()
  File "/home/isleof/.pyenv/versions/miniconda3-latest/envs/torch2/lib/python3.8/site-packages/pytorch_lightning/accelerators/gpu_accelerator.py", line 54, in train
    results = self.train_or_test()
  File "/home/isleof/.pyenv/versions/miniconda3-latest/envs/torch2/lib/python3.8/site-packages/pytorch_lightning/accelerators/accelerator.py", line 68, in train_or_test
    results = self.trainer.train()
  File "/home/isleof/.pyenv/versions/miniconda3-latest/env

TypeError: object of type 'NoneType' has no len()

<IPython.core.display.Javascript object>

## Inference

In [25]:
datamodule = NowcastingDataModule()
datamodule.setup("test")

final_preds = np.zeros((len(datamodule.test_dataset), 120, 120))

for fold in range(5):
    checkpoint = (
        args["model_dir"]
        / f"unet_fold{fold}_bs{args['batch_size']}_epoch{args['max_epochs']}_{args['optimizer']}_{args['scheduler']}.ckpt"
    )
    print(checkpoint)
    model = UNet.load_from_checkpoint(str(checkpoint))
    model.cuda()
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in tqdm(datamodule.test_dataloader()):
            batch = batch.cuda()
            imgs = model(batch)
            imgs = imgs.detach().cpu().numpy()
            imgs = imgs[:, 0, 4:124, 4:124]
            imgs = args["rng"] * imgs
            imgs = imgs.clip(0, 255)
            imgs = imgs.round()
            preds.append(imgs)

    preds = np.concatenate(preds)
    preds = preds.astype(np.uint8)
    final_preds += preds

    del model
    gc.collect()
    torch.cuda.empty_cache()

final_preds = final_preds.astype(np.uint8)
final_preds = final_preds.reshape(-1, 14400)

../models/unet_fold0_bs256_epoch50_adamw_cosine.ckpt



../models/unet_fold1_bs256_epoch50_adamw_cosine.ckpt



../models/unet_fold2_bs256_epoch50_adamw_cosine.ckpt



../models/unet_fold3_bs256_epoch50_adamw_cosine.ckpt



../models/unet_fold4_bs256_epoch50_adamw_cosine.ckpt


<IPython.core.display.Javascript object>

In [26]:
test_paths = datamodule.test_dataset.paths
test_filenames = [path.name for path in test_paths]

<IPython.core.display.Javascript object>

In [27]:
subm = pd.DataFrame({"file_name": test_filenames})
for i in tqdm(range(14400)):
    subm[str(i)] = final_preds[:, i]

<IPython.core.display.Javascript object>

In [28]:
subm.to_csv(
    f"unet_bs{args['batch_size']}_epoch{args['max_epochs']}_{args['optimizer']}_{args['scheduler']}.csv",
    index=False,
)
subm.head()

,file_name,0,1,2,3,4,5,6,7,8,...,14390,14391,14392,14393,14394,14395,14396,14397,14398,14399
0,test_00000.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,test_00001.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,test_00002.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,test_00003.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,test_00004.npy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<IPython.core.display.Javascript object>